## Case Study Background
The marketing team is having trouble joining 2 customer datasets from 2 different marketing campaigns as some user inputs were erroneous. They have identified that each customer from dataset 1 has exactly 1 match in dataset 2. You're tasked with performing data linkage to identify these matches.

### About the dataset

The datasets used in this worksheet is obtained from the `recordlinkage` library. It contains 2 dataframes of size 500, consisting of 8 personal detail fields. The record with index `rec-0-org` from dataset 1 has a match with the record with index `rec-0-dup-0` from dataset 2; the record with index `rec-1-org` from dataset 1 has a match with the record with index `rec-1-dup-0` from dataset 2; and so on...

For the sake of learning, we will pretend that we don't know that these are matches, and will see how many of these matches are returned by the linkage algorithms.

## Learning objectives
- Understand the concept of data linkage, blocking
- Know how to evaluate the performance of a linkage algorithm
- Know how to improve the performance of a linkage algorithm via blocking
- Know how to perform basic blocking using the `recordlinkage` library

## Workshop Overview
- Investigate the functions to generate the index pairs (exhaustive matching and blocking)
- Investigate the function to compare the records
- Calculate the performance metrics of a linkage algorithm
- Compare the exhaustive matching vs blocked matching
- Create new blocking features with feature engineering
- Answer theoretical questions related to the concepts practiced in the case study

## Acknowledgement / References
https://recordlinkage.readthedocs.io/en/latest/notebooks/link_two_dataframes.html

In [1]:
import pandas as pd
import recordlinkage
import time
# pip install recordlinkage

In [2]:
# Load toy datasets
sample1 = pd.read_csv('datalinkage_sample1.csv').set_index('rec_id')
sample2 = pd.read_csv('datalinkage_sample2.csv').set_index('rec_id')
sample1['date_of_birth'] = sample1['date_of_birth'].astype(str)
sample2['date_of_birth'] = sample2['date_of_birth'].astype(str)
sample1['postcode'] = sample1['postcode'].astype(str)
sample2['postcode'] = sample2['postcode'].astype(str)
display(sample1)
display(sample2)

,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth
rec_id,,,,,,,,
rec-0-org,flynn,rokobaro,12,herschell circuit,lawrence,2227,nsw,19720812
rec-1-org,karli,alderson,144,nulsen circuit,tingalpa,3139,nsw,19510826


,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth
rec_id,,,,,,,,
rec-0-dup-0,thomas,rokobaro,12,herschell circuit,lawrence,2272,nsw,nan
rec-1-dup-0,karli,alderson,144,nulsen circuit,tings lpa,3139,nsw,19510826.0


## <u> Concept: Data Linkage </u>

A data linkage / record matching pipeline between dataset A and B usually follows 4 steps:

1. Pre-process A and B & determine the features/attributes to calculate the similarity score
2. Generate a list of (indexA, indexB) to compare against one another. The strategy to generate the list can either be exhaustive (every record in A against every record in B); random; or via blocking.
3. For each (indexA, indexB), calculate the similarity score
4. Choose a threshold. If the score is >= this threshold, then indexA and indexB is a match.

For this tutorial, both datasets have been pre-processed, so we can skip step 1.

### Step 2: Generate the index pairs

In [3]:
def get_exhaustive_pairs(dfA, dfB):
    
    indexer = recordlinkage.Index()
    indexer.full()
    return indexer.index(dfA, dfB)

exhaustive_sample = get_exhaustive_pairs(sample1, sample2)
print(exhaustive_sample)
print('Number of pairs to perform matching:', len(exhaustive_sample))

MultiIndex([('rec-0-org', 'rec-0-dup-0'),
            ('rec-0-org', 'rec-1-dup-0'),
            ('rec-1-org', 'rec-0-dup-0'),
            ('rec-1-org', 'rec-1-dup-0')],
           names=['rec_id_1', 'rec_id_2'])
Number of pairs to perform matching: 4


### Step 3: Calculate the similarity score

In [4]:
def score_sim(index_pairs, dfA, dfB):
    
    """
    Perform linkage based on the index_pairs generated in step 2.
    """
    
    # Instantiate the scorer
    scorer = recordlinkage.Compare()
    
    # Define the scoring function for each feature. See https://recordlinkage.readthedocs.io/en/latest/ref-compare.html#module-recordlinkage.compare
    scorer.string('given_name', 'given_name', method='levenshtein', threshold=0.85, label='given_name')
    scorer.string('surname', 'surname', method='levenshtein', threshold=0.85, label='surname')
    scorer.string('date_of_birth', 'date_of_birth',  method='levenshtein', threshold=0.75, label='date_of_birth')
    scorer.exact('suburb', 'suburb', label='suburb')
    scorer.exact('state', 'state', label='state')
    scorer.string('postcode', 'postcode', label='postcode')
    scorer.string('address_1', 'address_1', method='levenshtein', threshold=0.85, label='address_1')
    
    # Perform the scoring and get the runtime for performance evaluation
    start = time.time()
    pairwise_sim = scorer.compute(index_pairs, dfA, dfB)
    end = time.time()
    runtime = end- start
    
    return pairwise_sim, runtime

In [5]:
pairwise_sim, run_time = score_sim(exhaustive_sample, sample1, sample2)
print('Matching time (secs):', run_time)

Matching time (secs): 0.013808012008666992


### Step 4: Compare against threshold and determining the matches

In [6]:
# Investigate the output
pairwise_sim

given_name  surname  date_of_birth  suburb  state  \
rec_id_1  rec_id_2                                                         
rec-0-org rec-0-dup-0         0.0      1.0            0.0       1      1   
          rec-1-dup-0         0.0      0.0            0.0       0      1   
rec-1-org rec-0-dup-0         0.0      0.0            0.0       0      1   
          rec-1-dup-0         1.0      1.0            1.0       0      1   

                       postcode  address_1  
rec_id_1  rec_id_2                          
rec-0-org rec-0-dup-0       0.5        1.0  
          rec-1-dup-0       0.0        0.0  
rec-1-org rec-0-dup-0       0.0        0.0  
          rec-1-dup-0       1.0        1.0

In [7]:
# Row sum and sort
pairwise_sim.sum(axis=1).sort_values(ascending=False)

rec_id_1   rec_id_2   
rec-1-org  rec-1-dup-0    6.0
rec-0-org  rec-0-dup-0    4.5
           rec-1-dup-0    1.0
rec-1-org  rec-0-dup-0    1.0
dtype: float64

In [8]:
def get_match(pairwise_sim, min_score=4):
    scores = pairwise_sim.sum(axis=1)
    return scores[scores>=min_score].index

matched_results = get_match(pairwise_sim)
matched_results

MultiIndex([('rec-0-org', 'rec-0-dup-0'),
            ('rec-1-org', 'rec-1-dup-0')],
           names=['rec_id_1', 'rec_id_2'])

# Try it on the full dataset

In [9]:
dataset1 = pd.read_csv('datalinkage_linkage1.csv').set_index('rec_id') # 500 rows
dataset2 = pd.read_csv('datalinkage_linkage2.csv').set_index('rec_id') # 500 rows

In [10]:
dataset1['date_of_birth'] = dataset1['date_of_birth'].astype(str)
dataset2['date_of_birth'] = dataset2['date_of_birth'].astype(str)
dataset1['postcode'] = dataset1['postcode'].astype(str)
dataset2['postcode'] = dataset2['postcode'].astype(str)

In [11]:
# Step 2
exhaustive = get_exhaustive_pairs(dataset1, dataset2)
print('Number of pairs to perform matching:', len(exhaustive))
# Step 3
pairwise_sim, run_time_full = score_sim(exhaustive, dataset1, dataset2)
print('Matching time (secs):', run_time_full)

Number of pairs to perform matching: 250000
Matching time (secs): 8.873044967651367


In [12]:
# Step 4
matched_results_full = get_match(pairwise_sim, min_score=3)
print('Number of matches found:', len(matched_results_full))

Number of matches found: 503


# Evaluation


In [13]:
# Get the true matches for this linkage dataset (courtesy of https://recordlinkage.readthedocs.io/en/latest/notebooks/link_two_dataframes.html)
from recordlinkage.datasets import load_febrl1
loaded_links = load_febrl1(return_links=True)[1]
# the order within each tuple is not consistent, so we fix this
links = []
for idx1, idx2 in loaded_links:
    links.append((idx2, idx1) if 'dup' in idx1 else (idx1, idx2))
links = pd.MultiIndex.from_tuples(links)
#links

In [14]:
def check_performance(matched_results, true_matches=links, total=250000):
    # total is the number of exhaustive pairings (500 * 500 = 250000)
    return { 
            'confusion_matrix': recordlinkage.confusion_matrix(true_matches, matched_results, total),
            'recall': recordlinkage.recall(true_matches, matched_results), 
            'precision': recordlinkage.precision(true_matches, matched_results),
            'accuracy':recordlinkage.accuracy(true_matches, matched_results, total)
           }

In [15]:
for metric, value in check_performance(matched_results_full).items():
    print(metric,'\n', value)

confusion_matrix 
 [[   500      0]
 [     3 249497]]
recall 
 1.0
precision 
 0.9940357852882704
accuracy 
 0.999988


/usr/lib/python3.10/site-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  return len(links_true & links_pred)
/usr/lib/python3.10/site-packages/recordlinkage/measures.py:179: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  return int(total) - len(links_true | links_pred)
/usr/lib/python3.10/site-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  return len(links_true & links_pred)
/usr/lib/python3.10/site-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprec

In [16]:
print('Reduction ratio: ', recordlinkage.reduction_ratio(exhaustive, [dataset1, dataset2]))

Reduction ratio:  0.0


<blockquote style="padding: 10px; background-color: #ebf5fb;">

    
## Discussion questions
1. How do we interpret the confusion matrix?
2. What is the reduction ratio? What does it tell us about the current indexing strategy?
3. How did our matching criteria perform?
4. How can we reduce the run time without negatively impacting the current performance?

## <u> Concept: Blocking </u>


Change step 2 function to do blocking on a specific column. From the documentation

> The argument `given_name` is the blocking variable. This variable has to be the name of a column in dfA and dfB. It is possible to parse a list of columns names to block on multiple variables. Blocking on multiple variables will reduce the number of record pairs even further.


In [17]:
def get_blocking_pairs(blocking_features, dfA, dfB):
    """
    Create an indexing strategy based on a list of `blocking_features`
    """
    indexer = recordlinkage.Index()
    for feature in blocking_features:
        indexer.block(feature)
    return indexer.index(dfA, dfB)

In [18]:
given_name_blocking = get_blocking_pairs(['given_name'], dataset1, dataset2)
print('Number of pairs to perform matching:', len(given_name_blocking))

pairwise_sim, given_name_run_time = score_sim(given_name_blocking, dataset1, dataset2)
print('Matching time (secs):', given_name_run_time)

matched_results = get_match(pairwise_sim, min_score=3)
print('Number of matches found (max = 500):', len(matched_results))

for metric, value in check_performance(matched_results).items():
    print(metric,'\n', value)

/usr/lib/python3.10/site-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  return len(links_true & links_pred)
/usr/lib/python3.10/site-packages/recordlinkage/measures.py:179: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  return int(total) - len(links_true | links_pred)
/usr/lib/python3.10/site-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  return len(links_true & links_pred)
/usr/lib/python3.10/site-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprec

Number of pairs to perform matching: 1181
Matching time (secs): 0.05480313301086426
Number of matches found (max = 500): 327
confusion_matrix 
 [[   326    174]
 [     1 249499]]
recall 
 0.652
precision 
 0.9969418960244648
accuracy 
 0.9993


/usr/lib/python3.10/site-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  return len(links_true & links_pred)
/usr/lib/python3.10/site-packages/recordlinkage/measures.py:179: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  return int(total) - len(links_true | links_pred)


In [19]:
print('Reduction ratio: ', recordlinkage.reduction_ratio(given_name_blocking, [dataset1, dataset2]))

Reduction ratio:  0.995276


<blockquote style="padding: 10px; background-color: #ebf5fb;">

    
## Discussion question
Compared to the exhaustive matching strategy, accuracy of the blocking by given_name is similar, but it's much faster. Does this mean the performance of blocking is superior to the exhaustive matching?

Answer: In this case, it is worse, because even though accuracy is the same, the number of matches is reduced, and there are fewer true-positives.
    It is much faster but performance is also worse.
    Accuracy is not very useful because the number of false negative is huge.

<blockquote style="padding: 10px; background-color: #FFD392;">


## Exercise
Change some parameters, like `min_score`, `threshold` and `blocking_features` to see how it affects the performance of the matching algorithm

## Improving the Blocking Strategy

Another technique we can try to improve the naive blocking strategy, beside manipulating the `min_score`, `threshold` and `blocking_features` parameters, is to create some new blocking features to improve the chance of potential matches being compared. This is known as feature engineering.

In the following section, we will create 2 new blocking features: the prefixes of surname and given name.

In [20]:
import re

dataset1['surname_prefix'] = dataset1['surname'].apply(lambda x: re.findall(r'^.{1,4}', str(x))[0])
dataset2['surname_prefix'] = dataset2['surname'].apply(lambda x: re.findall(r'^.{1,4}', str(x))[0])
dataset1['given_name_prefix'] = dataset1['given_name'].apply(lambda x: re.findall(r'^.{1,4}', str(x))[0])
dataset2['given_name_prefix'] = dataset2['given_name'].apply(lambda x: re.findall(r'^.{1,4}', str(x))[0])

# Revision question: What is the regex pattern finding? Print the dataset1 and dataset2 to confirm.

In [21]:
## blocking on prefixes of surname and given_name
prefix_blocking = get_blocking_pairs(['surname_prefix', 'given_name_prefix'], dataset1, dataset2)

print('Number of pairs to perform matching:', len(prefix_blocking))

pairwise_sim, run_time_with_block = score_sim(prefix_blocking, dataset1, dataset2)
print('Matching time (secs):', run_time_with_block)

matched_results = get_match(pairwise_sim, min_score=3)
print('Number of matches found:', len(matched_results))

for metric, value in check_performance(matched_results).items():
    print(metric,'\n', value)

Number of pairs to perform matching: 2898
Matching time (secs): 0.12687373161315918
Number of matches found: 445
confusion_matrix 
 [[   442     58]
 [     3 249497]]
recall 
 0.884
precision 
 0.9932584269662922
accuracy 
 0.999756


/usr/lib/python3.10/site-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  return len(links_true & links_pred)
/usr/lib/python3.10/site-packages/recordlinkage/measures.py:179: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  return int(total) - len(links_true | links_pred)
/usr/lib/python3.10/site-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  return len(links_true & links_pred)
/usr/lib/python3.10/site-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprec

In [22]:
print('Reduction ratio: ', recordlinkage.reduction_ratio(prefix_blocking, [dataset1, dataset2]))

Reduction ratio:  0.988408


## Comparing no-blocking and this improved blocking


In [23]:
performance_full = check_performance(matched_results_full)
performance_with_block = check_performance(matched_results)
performance_full['run-time'] = run_time_full
performance_full['pair-count'] = len(exhaustive)
performance_with_block['run-time'] = run_time_with_block
performance_with_block['pair-count'] = len(prefix_blocking)

/usr/lib/python3.10/site-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  return len(links_true & links_pred)
/usr/lib/python3.10/site-packages/recordlinkage/measures.py:179: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  return int(total) - len(links_true | links_pred)
/usr/lib/python3.10/site-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  return len(links_true & links_pred)
/usr/lib/python3.10/site-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprec

In [24]:
print('Exhaustive comparison:')
print(f"{performance_full['confusion_matrix']}")
print(f"\tprecision: {performance_full['precision']}")
print(f"\trecall: {performance_full['recall']}")
print(f"\trun-time: {round(performance_full['run-time'], 3)}")
print(f"\tpair-count: {performance_full['pair-count']}")

print('\nWith improved blocking:')
print(f"{performance_with_block['confusion_matrix']}")
print(f"\tprecision: {performance_with_block['precision']}")
print(f"\trecall: {performance_with_block['recall']}")
print(f"\trun-time: {round(performance_with_block['run-time'], 3)}, {round(100*performance_with_block['run-time']/performance_full['run-time'], 2)}% of exhaustive comparison")
print(f"\tpair-count: {performance_with_block['pair-count']}, {round(100*performance_with_block['pair-count']/performance_full['pair-count'], 2)}% of exhaustive comparison")

Exhaustive comparison:
[[   500      0]
 [     3 249497]]
	precision: 0.9940357852882704
	recall: 1.0
	run-time: 8.873
	pair-count: 250000

With improved blocking:
[[   442     58]
 [     3 249497]]
	precision: 0.9932584269662922
	recall: 0.884
	run-time: 0.127, 1.43% of exhaustive comparison
	pair-count: 2898, 1.16% of exhaustive comparison


The following plots show how increasing the size of the datasets affect the run time performance of each strategy (complexity analysis)
![](./efficiency_noblocking.png)
![](./efficiency_blocking.png)

<blockquote style="padding: 10px; background-color: #ebf5fb;">

    
## Theoretical questions
Suppose you are conducting data linkage between two databases, one with $m$ records and the other with $n$ records (assume $m \le n$). Under a basic (exhaustive) approach, $m \times n$ record comparisons will be needed. Assuming that there are no duplicates:

1. Assume there are no duplicates. What is the maximum number of record matches? What is the corresponding number of non-matching comparisons required in this circumstance?

Now suppose a blocking method is employed, where each record is assigned to exactly one block. Assume this method results in $b$ number of blocks.

2. What is the smallest possible number of comparisons? What is the value of $b$ in this scenario?
3. What is the largest possible number of comparisons? What is the value of $b$ in this scenario?
4. What is the advantage with a large $b$? What is the advantage with a small $b$?

In practice, a record is assigned to more than one block and records are not evenly allocated to blocks (for example, `alissa kilmartin` is assigned to the block with `surname_prefix==kilm` and `given_name_prefix==alis`).
    
5. How would this affect your answer to question 4?
6. What is the advantage of records being assigned to multiple blocks?
    
We've seen how accuracy does not truly reflect the performance of a blocking algorithm. Instead, the confusion matrix can be used to evaluate the performance based on TP, TN, FP, FN.
    
7. It is desirable to minimise both FP and FN, but it may be diffcult for a blocking algorithm to minimise both simultaneously. Give an example application where minimising FP is more important than minimising FN. Give an example application where minimising FN is more important than minimising FP.
       
8. When matching sensitive string records, the data is often hashed into **bloom filters**. Explain how string matching is performed with bloom filters, and the pros and cons of using bloom filter-based comparison instead of exact string comparison.

Answer:
1. maximum number of record matches is m, number of non-matches is m* (n-1)
2. 2 blocks, block A contains all records from dataset A, block B contains all records from dataset B. Thus, there's no comparisons.
3. m x n comparisons when b=1 (all records from dataset A & B in the same block)
4. fast, inaccurate vs slow, accurate
5. Dominant block size is the bottleneck for effciency. So, even if b is large, the blocking can still lead to ineffcient record linkage Small b tends to correspond to ineffciency; but it does not automatically guarantee accuracy. If matches are all of blocks, accuracy can still be low.
6. To reduce the chance of matched pairs not allocated to a common block. Matched pairs may not have the same parts of the records in common, sometimes, a blocking key may miss a pair but captured by another blocking key.
7. 

More important to minimise FP: e.g. Centrelink - might want to avoid sending out debt recovery letters to people who don't actually owe a debt 

Minimise FN: e.g. Medical screening tests - Often used to flag people who may have a particular disease for further tests. Don't want a FN to
mean that people who actually do have the disease don't receive further tests and treatment. Also national security, don't want a FN to mean
that a potential terrorist plot is not followed up on.

In practice there's often a trade-off involving the resources available for
further investigations. We are willing to have a number of false positives
to ensure we're avoiding false negatives, but don't want so many false
positives that we can't manually screen and investigate them all.

# <u> Challenge question </u>

The following section will introduce you to a Python implementation of bloom filters, using the `mmh3` library to implement the MurmurHash (MurmurHash3) hashing function. Note that you can use other hashing functions for bloom filters.

In [25]:
import mmh3
from nltk.util import bigrams

def bloom_filter(string1, string2, I, k):
    
    """
    Return two I-length bloom filters for string1 and string2 using k MMH3 hash functions
    """
    
    # Create a string into a set of smaller strings (bigrams)
    bigrams1 = [''.join(e) for e in bigrams(string1)]
    bigrams2 = [''.join(e) for e in bigrams(string2)]
    
    # Initialize the bloom filters
    bloomfilter1 = [0]*I
    bloomfilter2 = [0]*I

    for i in range(k):
    # For each hashing function, apply it to each element of the bigram lists, and update the according index to 1
        for w1 in bigrams1:
            idx1 = mmh3.hash(w1, seed=i) % I # Question: Why use the modulo function here?
            bloomfilter1[idx1] = 1
        for w2 in bigrams2:
            idx2 = mmh3.hash(w2, seed=i) % I
            bloomfilter2[idx2] = 1

    return bloomfilter1, bloomfilter2

In [26]:
bloomfilter1, bloomfilter2 = bloom_filter('_SMITH_', '_SMYTH_', I = 30, k = 2)

print('Bloom filter for _SMITH_:', bloomfilter1)
print('Bloom filter for _SMYTH_:', bloomfilter2)

Bloom filter for _SMITH_: [0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1]
Bloom filter for _SMYTH_: [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]


Write a function `dice_sim_bloomfilter()` that uses the bloom filters returned by `bloom_filter()` to calculate the Dice similarity scores. Your function should return the following results:

    print(dice_sim_bloomfilter('_SMITH_', '_SMYTH_', I = 30, k = 2))
    0.6666666666666666
    print(dice_sim_bloomfilter('_SMITH_', '_SMITH_', I = 30, k = 2))
    1.0
    print(dice_sim_bloomfilter('_SMITH_', '_SMOTH_', I = 30, k = 2))
    0.7619047619047619
    print(dice_sim_bloomfilter('_SMOTH_', '_SMYTH_', I = 30, k = 2))
    0.6363636363636364

In [27]:
def dice_sim_bloomfilter(string1, string2, I, k):
    ...

Try running the function with different values of `I` and `k` to investigate the effect of these parameters on the similarity scores. What are the pros and cons of having high `I`? What are the pros and cons of having high `k`?